# 중고차 가격 예측
데이터 출처 : https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=vw.csv  
수정 데이터 출처 : https://www.datamanim.com/dataset/03_dataq/typetwo.html#id16  

x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv  
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv  
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv  
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv  

project file(.ipynb) : https://github.com/fa-ina-tic/report/blob/main/UsedCar.ipynb

In [2]:
import pandas as pd
import numpy as np

x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")

In [4]:
df = pd.concat([x_train, y_train['price']], axis=1)
print(df.shape)

(4960, 11)


In [5]:
target = ['brand', 'model', 'transmission', 'fuelType']

for c in target:
    %timeit x_train.groupby(c)['tax'].sum()

for c in target:
    x_train[c] = x_train[c].astype('category')
    %timeit x_train.groupby(c)['tax'].sum()
    x_train[c] = x_train[c].astype('object')

258 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
281 µs ± 2.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
272 µs ± 5.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
255 µs ± 1.78 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
131 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
128 µs ± 871 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
145 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
148 µs ± 555 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
# data 기본 정보 확인
# X : 9 Columns 4960 Rows Null = None
# Y : 2 Columns 4960 Rows Null = None
print(x_train.info())
print(y_train.info())

print(x_train.describe())
print(y_train.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4919 entries, 0 to 4959
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         4919 non-null   int64  
 1   brand         4919 non-null   object 
 2   model         4919 non-null   object 
 3   year          4919 non-null   int64  
 4   transmission  4919 non-null   object 
 5   mileage       4919 non-null   int64  
 6   fuelType      4919 non-null   object 
 7   tax           4919 non-null   float64
 8   mpg           4919 non-null   float64
 9   engineSize    4919 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 422.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4919 entries, 0 to 4959
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   carID   4919 non-null   int64
 1   price   4919 non-null   int64
dtypes: int64(2)
memory usage: 115.3 KB
None
              car

In [6]:
# 결측값
print(x_train.isnull().sum())
print(y_train.isnull().sum())

drop_idx = x_train[(x_train['engineSize'] == 0) & ((x_train['fuelType'] == 'Diesel') | (x_train['fuelType'] == 'Petrol'))].index 

x_train.drop(drop_idx, inplace=True)
y_train.drop(drop_idx, inplace=True)

print(x_train.shape)
print(y_train.shape)

carID           0
brand           0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64
carID    0
price    0
dtype: int64
(4948, 10)
(4948, 2)


In [7]:
# 이상값

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

cate_col = ['brand', 'model', 'transmission', 'fuelType']
con_col = ['year', 'mileage','tax', 'mpg', 'engineSize']

In [8]:
for i in cate_col:
    fig = px.histogram(x_train, x=x_train[i])
    fig.show()

In [9]:
drop_idx_2 = x_train[(x_train['transmission']=='Other')|(x_train['fuelType'] == 'Other')|(x_train['fuelType'] == 'Electric')].index 

x_train.drop(drop_idx_2, inplace=True)
y_train.drop(drop_idx_2, inplace=True)
print(x_train.shape) #(4919, 10)
print(y_train.shape) #(4919, 2)

(4919, 10)
(4919, 2)


In [10]:
for i in con_col:
    fig = px.histogram(x_train, x=x_train[i])
    fig.show()

In [11]:
for i in con_col:
    boxplot = px.box(x_train, y=x_train[i])
    boxplot.show()

In [28]:
print('Skewness')
for i in con_col:
    skewness = x_train[i].skew()
    print(i, ' :', skewness)

Skewness
year  : -2.224568700454366
mileage  : 1.7708206321848745
tax  : 1.6866624065985518
mpg  : 10.478050348864436
engineSize  : 1.1336844764001448


In [66]:
# right skewed
from scipy import stats

posi_skewed = con_col[1:]
nega_skewed = ['year']

# positive skewed data tuning
for i in posi_skewed:
    log = x_train[[i]].apply(lambda x : np.log1p(x))
    reci = x_train[[i]].apply(lambda x : 1/(x+1))
    sqrt = x_train[[i]].apply(lambda x : np.sqrt(x))
    boxcox = pd.Series(stats.boxcox(x_train[i]+1)[0])

    print(i)
    print("log :", log.skew()[0])
    print("reci :", reci.skew()[0])
    print("sqrt :", sqrt.skew()[0])
    print("boxcox :", boxcox.skew())
    print('\n')


mileage
log : -1.8911438637608855
reci : 15.11030995066361
sqrt : 0.32851411105576417
boxcox : -0.15410112216268432


tax
log : -3.077816525449073
reci : 6.506662100677004
sqrt : -0.636287592242904
boxcox : 0.1896214824669765


mpg
log : 1.1122319467150998
reci : 15.1670460062778
sqrt : 5.491025006627867
boxcox : -0.2885061753976255


engineSize
log : -0.021737335044392698
reci : 2.013177307517078
sqrt : -0.09155791826648182
boxcox : 0.002417984716342038




In [70]:
# negative skewed data tuning
year_skew = - x_train[['year']] + 4000
log = year_skew.apply(lambda x : np.log1p(x))
reci = year_skew.apply(lambda x : 1/(x+1))
sqrt = year_skew.apply(lambda x : np.sqrt(x))
boxcox = pd.Series(stats.boxcox(year_skew['year']+1)[0])

print("year_skew")
print("log :", log.skew()[0])
print("reci :", reci.skew()[0])
print("sqrt :", sqrt.skew()[0])
print("boxcox :", boxcox.skew())

year_skew
log : 2.2138433807137408
reci : 0.0
sqrt : 2.2191975703222577
boxcox : 0


/opt/homebrew/lib/python3.9/site-packages/scipy/stats/_morestats.py:915: RuntimeWarning:

divide by zero encountered in log

/opt/homebrew/lib/python3.9/site-packages/scipy/optimize/_optimize.py:2321: RuntimeWarning:

invalid value encountered in double_scalars

/opt/homebrew/lib/python3.9/site-packages/scipy/optimize/_optimize.py:2322: RuntimeWarning:

invalid value encountered in double_scalars



In [86]:
for i in posi_skewed:
    boxcox = pd.DataFrame(stats.boxcox(x_train[i]+1)[0])
    fig = px.histogram(boxcox, width=600)
    fig.update_layout(xaxis_title=str(i), showlegend=False)
    box = px.box(boxcox, width = 600)
    box.update_layout(xaxis_title=str(i), showlegend=False)

    fig.show()
    box.show()

In [90]:
def get_outliers(df=None, column=None, weight=1.5):
    per_75 = np.percentile(df[column].values, 75)
    per_25 = np.percentile(df[column].values, 25)
    
    IQR = (per_75 - per_25) * weight
    high = per_75 + IQR
    low = per_25 - IQR
    
    outlier_idx = df[(df[column]>high)|(df[column]<low)].index
    return outlier_idx

for i in posi_skewed:
    boxcox = pd.DataFrame(stats.boxcox(x_train[i]+1)[0])
    boxcox.columns = [str(i)]

    outlier_idx = get_outliers(boxcox, i)
    number = len(outlier_idx)

    print(i, "outliers")
    print("Count :", number, '\n')

mileage outliers
Count : 4 

tax outliers
Count : 2249 

mpg outliers
Count : 179 

engineSize outliers
Count : 26 



In [93]:
kill_outlier = ['mileage', 'engineSize']

outlier_grid = ['tax', 'mpg']
IQR_grid = [2, 2.5, 3, 3.5, 4]

for i in outlier_grid:
    boxcox = pd.DataFrame(stats.boxcox(x_train[i]+1)[0])
    boxcox.columns = [str(i)]

    for j in IQR_grid:
        outlier_idx = get_outliers(boxcox, i, j)
        number = len(outlier_idx)

        print(i, "outliers, IQR :", j)
        print("Count :", number, '\n')

tax outliers, IQR : 2
Count : 2055 

tax outliers, IQR : 2.5
Count : 1992 

tax outliers, IQR : 3
Count : 1940 

tax outliers, IQR : 3.5
Count : 1940 

tax outliers, IQR : 4
Count : 1940 

mpg outliers, IQR : 2
Count : 67 

mpg outliers, IQR : 2.5
Count : 42 

mpg outliers, IQR : 3
Count : 39 

mpg outliers, IQR : 3.5
Count : 39 

mpg outliers, IQR : 4
Count : 27 

